## <span style=color:blue>Fetching GAEZ soil data for an ML pipeline</span>

###  <span style=color:blue>Note: rather than using GAEZ, you might want to use the ISRIC soil data. </span>

<span style=color:blue>E.g., see https://soilgrids.org and https://www.isric.org/explore/soilgrids/faq-soilgrids.  (It may take some digging around to make things work.  You probably want to use the EPSG:4326 (Plate Carre) projection.) Also, if you are working with yield data at the county level, then you might want to use soil grid data at the 5k x 5k gridsize, rather than then 1km x 1km or 250m x 250m gridsize. </span>

In [46]:
# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

### <span style=color:blue>Fetching the file state_county_lon_lat.csv which will be used below</span>

In [47]:
import pandas as pd

archive_dir = './lon_lat_data/'
scll_filename = 'state_county_lon_lat.csv'

df_scll = pd.read_csv(archive_dir + scll_filename)
print(df_scll.head())
print()
print(len(df_scll))


  state_name county_name         lon        lat
0     KANSAS    CHEYENNE -101.757549  39.795580
1     KANSAS     DECATUR -100.472769  39.794053
2     KANSAS      GRAHAM  -99.898062  39.340620
3     KANSAS      NORTON  -99.910003  39.794470
4     KANSAS     RAWLINS -101.099472  39.790480

557


### <span style=color:blue>Fetching several .tif files using urllib.</span>

<span style=color:blue>I selected this by visual inspection of various GAEZ data sets.  I was looking for data based on soil that appeared to differentiate parts of my region of interest.</span>
    
### <span style=color:blue>Note: for the 1st, 3rd and 4th tif files fetched below, the pixel values are categorical.  So we will have to use one-hot encodings for these</span>

In [48]:
import urllib

tif_dir = "./soil_data/"

url = {}

# using Land and Water Resources / Dominant AEZ class (33 classes) at 5 arc-minutes
# Based on 33 AEZ classes, even though pixel values are integer
url['AEZ_classes'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/aez/aez_v9v2red_5m_CRUTS32_Hist_8110_100_avg.tif"


# Using the URL of TIF file Soil Resources / Nutrient retention capacity, high inputs
# Based on 1 to 10, corresponding to bands 0.0 to 0.1; 0.1 to 02; etc.  So basically a numeric parameter
url['nutr_ret_high'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/soi1/SQ2_mze_v9aH.tif"

# using Land and Water Resources / Soil Resources / Most limiting soil quality rating factor, high inputs
# Based on 11 soil categories (and water), even though pixel values are integer
url['soil_qual_high'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/soi1/SQ0_mze_v9aH.tif"

# using Land and Water Resources / Soil Resources / Most limiting soil quality rating factor, low inputs
# same as previous
url['soil_qual_low'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/soi1/SQ0_mze_v9aL.tif"

# Leaving this one out because it is 43200 x 21600 pixels; don't want to work with different size input for now...
# using Land and Water Resources / Soil suitability, rain-fed, low-inputs
# url['soil_suit_rain_low'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/soi2/siLr_sss_mze.tif"

# using Suitability and Attainable Yield / Suitability Index / Suitability index range (0-10000);
#   within this chose crop = soybean; time period = 1981 to 2010; others empty
# this has numeric values from 0 to 10,000
url['suit_irrig_high_soy'] = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res05/CRUTS32/Hist/8110H/suHi_soy.tif"

urlkeys = url.keys()
print(urlkeys)

# Fetch the TIF files using the associated URLs

curr = curr_timestamp()
fileFullName = {}

# fetching the tif files from web and writing into local files
for k in urlkeys:
    fileFullName[k] = str(tif_dir + curr + '__' + k + '.tif')
    print(fileFullName[k])
    urllib.request.urlretrieve(url[k], fileFullName[k])
print(fileFullName)

dict_keys(['AEZ_classes', 'nutr_ret_high', 'soil_qual_high', 'soil_qual_low', 'suit_irrig_high_soy'])
./soil_data/2023-05-31_22-18-43__AEZ_classes.tif
./soil_data/2023-05-31_22-18-43__nutr_ret_high.tif
./soil_data/2023-05-31_22-18-43__soil_qual_high.tif
./soil_data/2023-05-31_22-18-43__soil_qual_low.tif
./soil_data/2023-05-31_22-18-43__suit_irrig_high_soy.tif
{'AEZ_classes': './soil_data/2023-05-31_22-18-43__AEZ_classes.tif', 'nutr_ret_high': './soil_data/2023-05-31_22-18-43__nutr_ret_high.tif', 'soil_qual_high': './soil_data/2023-05-31_22-18-43__soil_qual_high.tif', 'soil_qual_low': './soil_data/2023-05-31_22-18-43__soil_qual_low.tif', 'suit_irrig_high_soy': './soil_data/2023-05-31_22-18-43__suit_irrig_high_soy.tif'}


### <span style=color:blue>Fetching meta-data about the .tif files using GDAL, specifically the command-line operator gdalinfo.</span>

In [49]:
import json
import subprocess

def pull_useful(ginfo):  # should give as input the result.stdout from calling gdalinfo -json
    useful = {}
    useful['band_count'] = len(ginfo['bands'])
    # useful['cornerCoordinates'] = ginfo['cornerCoordinates']
    # useful['proj:transform'] = ginfo['stac']['proj:transform']
    useful['size'] = ginfo['size']
    # useful['bbox'] = ginfo['stac']['proj:projjson']['bbox']
    # useful['espgEncoding'] = ginfo['stac']['proj:epsg']
    return useful

gdalInfoReq = {}
gdalInfo = {}

useful = {}
for k in urlkeys:
    gdalInfoReq[k] = " ".join(["gdalinfo", "-json", fileFullName[k]])
    result = subprocess.run([gdalInfoReq[k]], shell=True, capture_output=True, text=True)
    print(result)
    gdalInfo[k] = json.loads(result.stdout)
    # if k == 'AEZ_classes':
    #     print(json.dumps(gdalInfo[k], indent=2, sort_keys=True))

    useful[k] = pull_useful(gdalInfo[k])
    print('\n', k)
    print(json.dumps(useful[k], indent=2, sort_keys=True))
 


CompletedProcess(args=['gdalinfo -json ./soil_data/2023-05-31_22-18-43__AEZ_classes.tif'], returncode=0, stdout='{\n  "description":"./soil_data/2023-05-31_22-18-43__AEZ_classes.tif",\n  "driverShortName":"GTiff",\n  "driverLongName":"GeoTIFF",\n  "files":[\n    "./soil_data/2023-05-31_22-18-43__AEZ_classes.tif"\n  ],\n  "size":[\n    4320,\n    2160\n  ],\n  "coordinateSystem":{\n    "wkt":"GEOGCRS[\\"WGS 84\\",\\n    ENSEMBLE[\\"World Geodetic System 1984 ensemble\\",\\n        MEMBER[\\"World Geodetic System 1984 (Transit)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G730)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G873)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G1150)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G1674)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G1762)\\"],\\n        MEMBER[\\"World Geodetic System 1984 (G2139)\\"],\\n        ELLIPSOID[\\"WGS 84\\",6378137,298.257223563,\\n            LENGTHUNIT[\\"metre\\",1]],\\n     

<span style=color:blue>Function to pull value from a pixel.  (Thanks to Claudio Spiess)   </span>

In [50]:
# following https://gis.stackexchange.com/a/299791, adapted to fetch just one pixel

import rasterio

def get_coordinate_pixel(tiff_file, lon, lat):
    dataset = rasterio.open(tiff_file)
    py, px = dataset.index(lon, lat)
    # create 1x1px window of the pixel
    window = rasterio.windows.Window(px, py, 1, 1)
    # read rgb values of the window
    clip = dataset.read(window=window)
    # print(clip)
    return clip[0][0][0]

# testing the function
tiff_file = fileFullName['AEZ_classes']
# tiff_file = '/Users/rick/AG-CODE--v03/GAEZ-SOIL-for-ML/OUTPUTS/2023-05-20_23-09-36__AEZ_classes.tif'
print(df_scll.iloc[[0]])
test_lon = df_scll.iloc[0]['lon']
test_lat = df_scll.iloc[0]['lat']
print(test_lon, test_lat, type(test_lon), type(test_lat))
print()

val = get_coordinate_pixel(tiff_file, test_lon, test_lat)
print(type(val))
print(val)

  state_name county_name         lon       lat
0     KANSAS    CHEYENNE -101.757549  39.79558
-101.7575491 39.7955799 <class 'numpy.float64'> <class 'numpy.float64'>

<class 'numpy.uint8'>
16


## <span style=color:blue>Now adding all 5 soil values to the rows of df_scll.  This takes a while to run. </span>

<span style=color:blue>With the rasterio function, the retrieved values are of type int.  If using the gdalinfo-based fucntion, then the values coming from the XML are all strings, one should convert to int when loading into the new df that we are building</span>

In [51]:
df3 = df_scll.copy()
print(df3.head())
print(len(df3))

for k in urlkeys:
#     df3[k] = df3.apply(lambda r: fetch_tif_value(r['lon'], r['lat'], k, False), axis=1)
    tiff_file = fileFullName[k]
    df3[k] = df3.apply(lambda r: get_coordinate_pixel(tiff_file, r['lon'], r['lat']), axis=1)
    
print()
print(df3.head())
print(len(df3))

  state_name county_name         lon        lat
0     KANSAS    CHEYENNE -101.757549  39.795580
1     KANSAS     DECATUR -100.472769  39.794053
2     KANSAS      GRAHAM  -99.898062  39.340620
3     KANSAS      NORTON  -99.910003  39.794470
4     KANSAS     RAWLINS -101.099472  39.790480
557

  state_name county_name         lon        lat  AEZ_classes  nutr_ret_high   
0     KANSAS    CHEYENNE -101.757549  39.795580           16             10  \
1     KANSAS     DECATUR -100.472769  39.794053           16             10   
2     KANSAS      GRAHAM  -99.898062  39.340620           16             10   
3     KANSAS      NORTON  -99.910003  39.794470           16             10   
4     KANSAS     RAWLINS -101.099472  39.790480           16             10   

   soil_qual_high  soil_qual_low  suit_irrig_high_soy  
0               9              9                10000  
1               9              9                 8536  
2               9              9                 7778  
3       

Get unique values for 'AEZ classes', 'soil_qual_high', and 'soil_qual_low' for later 1 hot encoding

In [52]:
AEZ_classes_unique_values = []
soil_qual_high_unique_values = []
soil_qual_low_unique_values = []

for k in urlkeys:
    if k == "AEZ_classes":
        for index, row in df3[[k]].drop_duplicates().iterrows():
            AEZ_classes_unique_values.append(row['AEZ_classes'])
    elif k == "soil_qual_high":
        for index, row in df3[[k]].drop_duplicates().iterrows():
            soil_qual_high_unique_values.append(row['soil_qual_high'])
    elif k == "soil_qual_low":
        for index, row in df3[[k]].drop_duplicates().iterrows():
            soil_qual_low_unique_values.append(row['soil_qual_low'])
    
    #print(df3[[k]].drop_duplicates().head(100), end="\n\n")

AEZ_classes_unique_values.sort()
soil_qual_high_unique_values.sort()
soil_qual_low_unique_values.sort()

print("AEZ")
print(AEZ_classes_unique_values)
print("soil qual high")
print(soil_qual_high_unique_values)
print("soil qual low")
print(soil_qual_low_unique_values)

AEZ
[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 29, 32, 33]
soil qual high
[2, 3, 4, 5, 6, 7, 8, 9, 10, 13]
soil qual low
[2, 3, 4, 5, 6, 7, 8, 9, 10, 13]


## <span style=color:blue>Replacing the columns for 'AEZ_classes', 'soil_qual_high', 'soil_qual_low' with multiple "1-hot" columns.   (We could also use the OneHotEncoder from scikit, but I'm choosing to do it here and now on the raw data.)</span>

<span style=color:blue>Following https://stackoverflow.com/questions/37292872/how-can-i-one-hot-encode-in-python</span>



In [53]:
df4 = df3.copy()

# Get one hot encoding of columns 'AEZ-classes'
one_hot = pd.get_dummies(df4['AEZ_classes'])
# Drop original as it is now encoded
df4 = df4.drop('AEZ_classes',axis = 1)
# Join the encoded df
df4 = df4.join(one_hot)

print(len(df4))
print(df4.head())
print(df4.columns.tolist())
# output was ['state_name', 'county_name', 'lon', 'lat', 'nutr_ret_high', 
#             'soil_qual_high', 'soil_qual_low', 'suit_irrig_high_soy', 
#              16, 17, 18, 19, 20, 21, 27, 28, 32]

cols = {}
for count, value in enumerate(AEZ_classes_unique_values):
    cols[value] = "AEZ_" + str(count+1)
# cols = { 7: 'AEZ_1',
#          8: 'AEZ_2',
#          9: 'AEZ_3',
#          11: 'AEZ_4',
#          20: 'AEZ_5',
#          21: 'AEZ_6',
#          27: 'AEZ_7',
#          28: 'AEZ_8',
#          32: 'AEZ_9',
#          32: 'AEZ_9',
#          32: 'AEZ_9',
#          32: 'AEZ_9',
#          32: 'AEZ_9',}
df4 = df4.rename(columns=cols)
print(df4.columns.tolist())
print(df4.head())



557
  state_name county_name         lon        lat  nutr_ret_high   
0     KANSAS    CHEYENNE -101.757549  39.795580             10  \
1     KANSAS     DECATUR -100.472769  39.794053             10   
2     KANSAS      GRAHAM  -99.898062  39.340620             10   
3     KANSAS      NORTON  -99.910003  39.794470             10   
4     KANSAS     RAWLINS -101.099472  39.790480             10   

   soil_qual_high  soil_qual_low  suit_irrig_high_soy      7      8  ...   
0               9              9                10000  False  False  ...  \
1               9              9                 8536  False  False  ...   
2               9              9                 7778  False  False  ...   
3               9              9                 7778  False  False  ...   
4               9              9                10000  False  False  ...   

      18     19     20     25     26     27     28     29     32     33  
0  False  False  False  False  False  False  False  False  False  Fa

In [54]:
# making a copy of df4, because may run this cell a couple of times as I develop it
df5 = df4.copy()

# Get one hot encoding of columns 'soil_qual_high'
one_hot1 = pd.get_dummies(df5['soil_qual_high'])
# Drop original as it is now encoded
df5 = df5.drop('soil_qual_high',axis = 1)
# Join the encoded df
df5 = df5.join(one_hot1)

print(len(df5))
print(df5.head())
print(df5.columns.tolist())
# output was ['state_name', 'county_name', 'lon', 'lat', 'nutr_ret_high', 
#             'soil_qual_low', 'suit_irrig_high_soy', 'AEZ_1', 'AEZ_2', 'AEZ_3', 
#             'AEZ_4', 'AEZ_5', 'AEZ_6', 'AEZ_7', 'AEZ_8', 'AEZ_9', 
#             4, 5, 6, 7, 8, 9, 10]

print()
cols = {}
for count, value in enumerate(soil_qual_high_unique_values):
    cols[value] = "SQH_" + str(count+1)
# cols = { 4: 'SQH_1',
#          5: 'SQH_2',
#          6: 'SQH_3',
#          7: 'SQH_4',
#          8: 'SQH_5',
#          9: 'SQH_6',
#          10: 'SQH_7'}
df5 = df5.rename(columns=cols)
print(df5.columns.tolist())
print(df5.head())

557
  state_name county_name         lon        lat  nutr_ret_high  soil_qual_low   
0     KANSAS    CHEYENNE -101.757549  39.795580             10              9  \
1     KANSAS     DECATUR -100.472769  39.794053             10              9   
2     KANSAS      GRAHAM  -99.898062  39.340620             10              9   
3     KANSAS      NORTON  -99.910003  39.794470             10              9   
4     KANSAS     RAWLINS -101.099472  39.790480             10              9   

   suit_irrig_high_soy  AEZ_1  AEZ_2  AEZ_3  ...      2      3      4      5   
0                10000  False  False  False  ...  False  False  False  False  \
1                 8536  False  False  False  ...  False  False  False  False   
2                 7778  False  False  False  ...  False  False  False  False   
3                 7778  False  False  False  ...  False  False  False  False   
4                10000  False  False  False  ...  False  False  False  False   

       6      7      8     9

In [55]:
# making a copy of df5, because may run this cell a couple of times as I develop it
df6 = df5.copy()

# Get one hot encoding of columns 'soil_qual_low'
one_hot2 = pd.get_dummies(df6['soil_qual_low'])
# Drop original as it is now encoded
df6 = df6.drop('soil_qual_low',axis = 1)
# Join the encoded df
df6 = df6.join(one_hot2)

print(len(df6))
print(df6.head())
print(df6.columns.tolist())
# output was ['state_name', 'county_name', 'lon', 'lat', 'nutr_ret_high', 
#             'suit_irrig_high_soy', 'AEZ_1', 'AEZ_2', 'AEZ_3', 'AEZ_4', 
#             'AEZ_5', 'AEZ_6', 'AEZ_7', 'AEZ_8', 'AEZ_9', 
#             'SQH_1', 'SQH_2', 'SQH_3', 'SQH_4', 'SQH_5', 'SQH_6', 'SQH_7', 
#              4, 5, 6, 7, 8, 9, 10]

print()
cols = {}
for count, value in enumerate(soil_qual_low_unique_values):
    cols[value] = "SQL_" + str(count+1)
# cols = { 4: 'SQL_1',
#          5: 'SQL_2',
#          6: 'SQL_3',
#          7: 'SQL_4',
#          8: 'SQL_5',
#          9: 'SQL_6',
#          10: 'SQL_7'}
df6 = df6.rename(columns=cols)
print(df6.columns.tolist())
print(df6.head())

557
  state_name county_name         lon        lat  nutr_ret_high   
0     KANSAS    CHEYENNE -101.757549  39.795580             10  \
1     KANSAS     DECATUR -100.472769  39.794053             10   
2     KANSAS      GRAHAM  -99.898062  39.340620             10   
3     KANSAS      NORTON  -99.910003  39.794470             10   
4     KANSAS     RAWLINS -101.099472  39.790480             10   

   suit_irrig_high_soy  AEZ_1  AEZ_2  AEZ_3  AEZ_4  ...      2      3      4   
0                10000  False  False  False  False  ...  False  False  False  \
1                 8536  False  False  False  False  ...  False  False  False   
2                 7778  False  False  False  False  ...  False  False  False   
3                 7778  False  False  False  False  ...  False  False  False   
4                10000  False  False  False  False  ...  False  False  False   

       5      6      7      8     9     10     13  
0  False  False  False  False  True  False  False  
1  False  Fals

<span style=color:blue>Archiving this df     </span>

In [56]:
archives_dir = './soil_data/'
filename = 'state_county_lon_lat_soil.csv'
df6.to_csv(archives_dir + filename, index=False)
print('wrote file: ', archives_dir + filename)

wrote file:  ./soil_data/state_county_lon_lat_soil.csv


### <span style=color:blue>Comparing the pixel value obtained by the rasterio-based function with pixel value obtained by using gdalinfo and then extraction from the XML that is returned   </span>

In [57]:
import xml.etree.ElementTree as ET

# converting from lat/long into a pixel location for a global tif with size 4320x2160
def convert_to_pix(lon, lat):
    x = round((lon + 180) * 12)
    y = round((90 - lat) * 12)
    return x,y

# recall: fullFileName[k] holds full dir + file name for key k
f = fileFullName['AEZ_classes']
# f = '/Users/rick/AG-CODE--v03/GAEZ-SOIL-for-ML/OUTPUTS/2023-05-20_23-09-36__AEZ_classes.tif'

print(df_scll.iloc[[0]])
test_lon = df_scll.iloc[0]['lon']
test_lat = df_scll.iloc[0]['lat']
print(test_lon, test_lat, type(test_lon), type(test_lat))


x,y = convert_to_pix(test_lon, test_lat)

print(x,y)

# gdal terminal command to fectch pixel value
val = " ".join(['gdallocationinfo -xml', fileFullName['AEZ_classes'], str(x), str(y)])
print(val)

result = subprocess.run([val], 
                         shell=True, capture_output=True,text=True)
print(result.stdout)
print(result.stderr)

root = ET.fromstring(result.stdout)
val_out = root[0][0].text
print(val_out)




  state_name county_name         lon       lat
0     KANSAS    CHEYENNE -101.757549  39.79558
-101.7575491 39.7955799 <class 'numpy.float64'> <class 'numpy.float64'>
939 602
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__AEZ_classes.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>16</Value>
  </BandReport>
</Report>


16


<span style=color:blue>Now checking the structure of the outputs from each of the 5 .tif files.  Each of them has just 1 band, and so the rasterio-based function above will work</span>

In [58]:
# still working with the location of first county -- in df_scll.iloc[[0]]

for k in urlkeys:
    print()
    print(k)
    
    val = " ".join(['gdallocationinfo -xml', fileFullName[k], str(x), str(y)])
    print(val)

    result = subprocess.run([val], 
                             shell=True, capture_output=True,text=True)
    print(result.stdout)
    print(result.stderr)

    root = ET.fromstring(result.stdout)
    val_out = root[0][0].text
    print(val_out)






AEZ_classes
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__AEZ_classes.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>16</Value>
  </BandReport>
</Report>


16

nutr_ret_high
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__nutr_ret_high.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>10</Value>
  </BandReport>
</Report>


10

soil_qual_high
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__soil_qual_high.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>9</Value>
  </BandReport>
</Report>


9

soil_qual_low
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__soil_qual_low.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>9</Value>
  </BandReport>
</Report>


9

suit_irrig_high_soy
gdallocationinfo -xml ./soil_data/2023-05-31_22-18-43__suit_irrig_high_soy.tif 939 602
<Report pixel="939" line="602">
  <BandReport band="1">
    <Value>10000</Valu